In [ ]:
from ipynb.fs.full.BreastFunctions import *
#from BreastFunctions_py import *
import numpy as np
import pandas as pd

In [ ]:
# Prepare this dataframe (total results) only once
total_results_dataframe = pd.DataFrame(columns=["Patient to be predicted/left out" , "Metastasis-Free Subpopulation" , "Metastasis Subpopulation" , 
"Metastasis Status","Number of Clusters" , "SAM Features for each tSNE run on new subset"])

# Only load if needed to load total results from previous runs to continue on
#total_results_dataframe = pd.read_csv("total_breastCancer_results.csv")


In [ ]:
HE_image , MSI_data_cube , goodlist, peak_list, pixel_to_sample_ID, sample_only_data, sample_only_scaled_data , sample_ID_pixels = ReadBreastData()

In [ ]:
## Selecting Patient as test
testPatient_ID = 1
#-------------------------------------------------------

train_scaled_data,train_data,train_ID_pixels,test_scaled_data,test_data,test_ID_pixels = SplitData(ID = testPatient_ID ,
        sample_only_data = sample_only_data,
        sample_only_scaled_data = sample_only_scaled_data,
        sample_ID_pixels = sample_ID_pixels )


In [ ]:
tsne_results = tSNE(train_scaled_data)
np.save("patient_TSNE/BreastCancer_tSNE_Model_Patient_{}".format(testPatient_ID) , tsne_results)

In [ ]:
tsne_results = np.load("patient_TSNE/BreastCancer_tSNE_Model_Patient_{}.npy".format(testPatient_ID))

In [ ]:
kmeans_8 , kmeans_7, kmeans_6, kmeans_5 = KMeans_results(tsne_results)

In [ ]:
Clinical_data = ReadClinicalData(testPatient_ID , "ClinicalData.xlsx")
Clinical_data_full = pd.read_excel("ClinicalData.xlsx","Breast Cancer Clinical Data")

In [ ]:
# Returns Non-Metastasis and Metastasis clusters
state08, state18 = metastasis_status(kmeans_8.labels_, Clinical_data,train_ID_pixels)
state07, state17 = metastasis_status(kmeans_7.labels_, Clinical_data,train_ID_pixels)
state06, state16 = metastasis_status(kmeans_6.labels_, Clinical_data,train_ID_pixels)
state05, state15 = metastasis_status(kmeans_5.labels_, Clinical_data,train_ID_pixels)

# Plots bar plot of Non-Metastasis and Metastasis clusters
plot_metastasis_status(state08, state18)
plot_metastasis_status(state07, state17)
plot_metastasis_status(state06, state16)
plot_metastasis_status(state05, state15)

# Create dict to save kmeans results and select them later
kmeans_dict = {"8_Clusters": [kmeans_8.labels_,state08] , "7_Clusters": [kmeans_7.labels_,state07]  , "6_Clusters": [kmeans_6.labels_,state06] , "5_Clusters": [kmeans_5.labels_,state05] }

In [ ]:
# Select the correct kmeans that shows best results from metastasis analysis, must have one fully metastasis cluster
selected_clusters_value = 7

# ----------------------------------------------------------------------------------

# Detect the fully metastasis cluster, select the correct kmeans that shows best results from metastasis analysis
labels = kmeans_dict[str(selected_clusters_value)+"_Clusters"][0]
selected_state = kmeans_dict[str(selected_clusters_value)+"_Clusters"][1]

fully_metastasis_cluster_label = 0
for cluster in selected_state:
    if not cluster: # Cluster list is empty, has no healthy patients, so it's fully metastasis cluster
        break # fully_metastasis_cluster_label now has the right label, exit loop
    
    fully_metastasis_cluster_label += 1

print("Fully metastasis cluster label's index is {} and it is cluster {}".format(fully_metastasis_cluster_label,fully_metastasis_cluster_label+1))

In [ ]:
protein_dataframe = SAM_Analysis(labels, Clinical_data, train_ID_pixels, fully_metastasis_cluster_label, train_data, peak_list)
protein_dataframe.to_csv("BreastCancerAverageSpectrumValues.csv",index=False)

# Run R file in order to generate significant proteins using the csv file

In [ ]:
#Need to read significant protein file outputted from R

SAM_protein = readSignificantProteins("significant_breast_proteins.json",delete=False)

SAM_protein_indices=[]
for i in range(len(peak_list)):

    if peak_list[i][0] in SAM_protein:
        SAM_protein_indices.append(i)

train_scaled_selected_proteins_data = train_scaled_data[:,SAM_protein_indices]
test_scaled_selected_proteins_data = test_scaled_data[:,SAM_protein_indices]

train_selected_proteins_data = train_data[:,SAM_protein_indices]
test_selected_proteins_data = test_data[:,SAM_protein_indices]

train_labels = TargetLabelsCreation(labels , Clinical_data, train_ID_pixels, fully_metastasis_cluster_label)

In [ ]:
# Test rbf at c = 0.1 , 1 , 10
# Test linear at c = 0.1 , 1 , 10
y_pred_SVM = SVM("rbf" , 1 , train_scaled_data , train_labels , test_scaled_data)

In [ ]:
Metastasis_prob_SVM , Non_Metastasis_prob_SVM = ProbabilityCalc(y_pred_SVM)

In [ ]:
y_pred_KNN = KNN(5 , train_scaled_data, train_labels, test_scaled_data)

In [ ]:
Metastasis_prob_KNN , Non_Metastasis_prob_KNN = ProbabilityCalc(y_pred_KNN)

In [ ]:
# Selected Metastasis probabilities, select KNN or SVM results
Metastasis_prob = Metastasis_prob_KNN
Non_Metastasis_prob = Non_Metastasis_prob_KNN

In [ ]:
# Run after model complete to save the result in the total results dataframe
total_results_dataframe = OutputDataframe(total_results_dataframe, testPatient_ID , Clinical_data_full, Non_Metastasis_prob, Metastasis_prob, SAM_protein)

In [ ]:
total_results_dataframe

In [ ]:
# Only run if needed to save and load at a later time
total_results_dataframe.to_csv("total_breastCancer_results.csv" , index = False)